## 문장 분류 모델
- 문장을 분류하는 모델

In [1]:
from torch import nn

In [2]:
class SentenceClassifier(nn.Module):
    def __init__(self, n_vocab, hidden_dim, embedding_dim, n_layers, dropout=0.5, bidirectional=True, model_type="lstm"):
        super().__init__()  # 부모클래스 상속

        self.embedding = nn.Embedding(num_embeddings=n_vocab, embedding_dim=embedding_dim, padding_idx=0)

        # rnn모델 일 경우
        if model_type == 'rnn':
            self.model = nn.RNN(
                input_size=embedding_dim, hidden_size=hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout, batch_first=True
            )
        # lstm모델 일 경우
        elif model_type == 'lstm':
            self.model = nn.LSTM(
                input_size=embedding_dim, hidden_size=hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout, batch_first=True
            )

        # bidirectional은 양방향성을 의미하는 파라미터
        if bidirectional:
            self.classifier = nn.Linear(hidden_dim * 2,1)   # 양방향일때 타임스탭에서 양방향의 정보(순방향,역방향)의 출력들을 결합하여 분류기에 전달
        else:
            self.classifier = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        output, _ = self.model(embeddings) 
        last_output = output[:, -1, :]
        last_output = self.dropout(last_output)
        logits = self.classifier(last_output)
        return logits

[1] 모듈로딩 및 데이터셋 불러오기

In [3]:
import pandas as pd
from Korpora import Korpora

In [4]:
corpus = Korpora.load("nsmc")
corpus_df = pd.DataFrame(corpus.test)


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\user\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\user

In [5]:
# train, test 나누기
train = corpus_df.sample(frac=0.9, random_state=42) # 랜덤으로 90를 선택
test = corpus_df.drop(train.index)                  # 트레인을 삭제해서 비율을 train : test = 9: 1

print(train.head(5).to_markdown())
print('Training Data Size : ', len(train))
print("Testing Data Size : ", len(test))

|       | text                                                                                     |   label |
|------:|:-----------------------------------------------------------------------------------------|--------:|
| 33553 | 모든 편견을 날려 버리는 가슴 따뜻한 영화. 로버트 드 니로, 필립 세이모어 호프만 영원하라. |       1 |
|  9427 | 무한 리메이크의 소재. 감독의 역량은 항상 그 자리에...                                    |       0 |
|   199 | 신날 것 없는 애니.                                                                       |       0 |
| 12447 | 잔잔 격동                                                                                |       1 |
| 39489 | 오랜만에 찾은 주말의 명화의 보석                                                         |       1 |
Training Data Size :  45000
Testing Data Size :  5000


[2] 데이터 전처리

In [6]:
# 데이터 토큰화 및 단어 사전 구축
from konlpy.tag import Okt
from collections import Counter

## 단어사전을 만드는 함수
def build_vocab(corpus, n_vocab, special_tokens): #n_vocab은 최대 사용할 단어개수
    counter = Counter()
    for tokens in corpus:
        counter.update(tokens)
    vocab = special_tokens
    for token, count in counter.most_common(n_vocab): # 가장 많이 등장한 순서대로 상위 n_vocab개의 단어 반환
        vocab.append(token)
    return vocab



In [7]:
tokenizer = Okt()
train_tokens = [tokenizer.morphs(review) for review in train.text]   # 형태소 단위로 나눠서 리스트에 저장
test_tokens = [tokenizer.morphs(review) for review in test.text]

vocab = build_vocab(corpus=train_tokens, n_vocab=5000, special_tokens=["<pad>", "<unk>"])
token_to_id = {token: idx for idx, token in enumerate(vocab)} # 저장이 <pad>": 0, "<unk>": 1, "오늘": 2, "날씨": 3 이런식
id_to_token = {idx: token for idx, token in enumerate(vocab)} # {0: "<pad>", 1: "<unk>", 2: "오늘", 3: "날씨"

print(vocab[:10])
print(len(vocab))

['<pad>', '<unk>', '.', '이', '영화', '의', '..', '가', '에', '...']
5002


In [8]:
## 정수 인코딩 및 패딩
import numpy as np
# 패딩까지 해주는 함수
def pad_sequences(sequences, max_length, pad_value):  #입력받는 sequences는 리스트형태
    result = list()
    for sequence in sequences:
        sequence = sequence[:max_length] # 시퀀스가 max_lenth보다 길경우 뒷부분을 자른다
        pad_length = max_length - len(sequence)
        padded_sequence = sequence + [pad_value] * pad_length   # 시퀀스가 짧을경우 남은부분을 0으로 채움
        result.append(padded_sequence)
    return np.asarray(result)   

In [10]:
unk_id = token_to_id["<unk>"]   # 단어사전에 없는 단어를 처리할때
train_ids = [[token_to_id.get(token, unk_id) for token in review] for review in train_tokens]  # 각 토큰에 대응하는 정수 ID반환, 존재하지않으면 unk
test_ids = [[token_to_id.get(token, unk_id) for token in review] for review in test_tokens]

max_length = 32
pad_id = token_to_id["<pad>"]   # <pad>": 0, "<unk>": 인 사전이니까 0으로 패딩을 한다는 의미
train_ids = pad_sequences(train_ids, max_length, pad_id)
test_ids = pad_sequences(test_ids, max_length, pad_id)

print(train_ids[0])
print(test_ids[0])

# 결과로 모두 32에 맞춰져서 나온다

[ 223 1716   10 4036 2095  193  755    4    2 2330 1031  220   26   13
 4839    1    1    1    2    0    0    0    0    0    0    0    0    0
    0    0    0    0]
[3307    5 1997  456    8    1 1013 3906    5    1    1   13  223   51
    3    1 4684    6    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]


[3] 모델 학습 준비

In [11]:
## 데이터로더
import torch
from torch.utils.data import TensorDataset, DataLoader

train_ids = torch.tensor(train_ids)
test_ids = torch.tensor(test_ids)

train_labels = torch.tensor(train.label.values, dtype=torch.float32)
test_labels = torch.tensor(test.label.values, dtype=torch.float32)

train_dataset = TensorDataset(train_ids, train_labels)
test_dataset = TensorDataset(test_ids, test_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [12]:
for feature, label in train_loader:
    print(feature)
    print(label)
    break

tensor([[ 118,  355,    1,  754,   36,  152,   79,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [   1,    4,    5, 1879,    2,  612,   75,  410, 1476, 3909,    3,  164,
          285,    2, 2869,    7, 1370,   28, 1771,    4,    2,  113,  255,  380,
         3157,    3,  942,   98,  119,    0,    0,    0],
        [   1,    1,    1,   10,  926,    1,   10, 1458,   56,    1, 4874,   10,
            1, 1417,    2,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [ 148,  167,  106,  146,    8,    1,    1,    1, 1014,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [  61,    6,    1, 4998,   55,   39,  343,   60,  584,  285,    2,  192,
          162,    3,    1,    1,   61, 

In [14]:
# 문장 분류하는 모델
class SentenceClassifier(nn.Module):
    def __init__(self, n_vocab, hidden_dim, embedding_dim, n_layers, dropout=0.5, bidirectional=True, model_type="lstm"):
        super().__init__()  # 부모클래스 상속

        self.embedding = nn.Embedding(num_embeddings=n_vocab, embedding_dim=embedding_dim, padding_idx=0)

        # rnn모델 일 경우
        if model_type == 'rnn':
            self.model = nn.RNN(
                input_size=embedding_dim, hidden_size=hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout, batch_first=True
            )
        # lstm모델 일 경우
        elif model_type == 'lstm':
            self.model = nn.LSTM(
                input_size=embedding_dim, hidden_size=hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout, batch_first=True
            )

        # bidirectional은 양방향성을 의미하는 파라미터
        if bidirectional:
            self.classifier = nn.Linear(hidden_dim * 2,1)   # 양방향일때 타임스탭에서 양방향의 정보(순방향,역방향)의 출력들을 결합하여 분류기에 전달
        else:
            self.classifier = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        output, _ = self.model(embeddings) 
        last_output = output[:, -1, :]
        last_output = self.dropout(last_output)
        logits = self.classifier(last_output)
        return logits

In [13]:
## 손실 함수와 최적화 함수 정의
from torch import optim

n_vocab = len(token_to_id)   # 단어사전의 크기
hidden_dim = 64     # 은닉 사태의 크기
embedding_dim = 128   #임베딩 벡터의 차원 128차원으로 사용
n_layers = 2     # 2층

device = "cuda" if torch.cuda.is_available() else "cpu"
classifier = SentenceClassifier(n_vocab=n_vocab, hidden_dim=hidden_dim, embedding_dim=embedding_dim, n_layers=n_layers).to(device)
criterion = nn.BCEWithLogitsLoss().to(device)   # 2진분류 손실함수
optimizer = optim.RMSprop(classifier.parameters(), lr=0.001)

[4] 모델 학습

In [15]:
# 모델 학습하는 함수
def train(model, datasets, criterion, optimizer, device, interval):
    model.train()
    losses = list()

    for step, (input_ids, labels) in enumerate(datasets):
        input_ids = input_ids.to(device)
        labels = labels.to(device).unsqueeze(1)

        logits = model(input_ids)
        loss = criterion(logits, labels)
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step%interval == 0:
            print(f'Train Loss {step} : {np.mean(losses)}')

In [16]:
# 모델 테스트하는 함수
def test(model, datasets, criterion, device):
    model.eval()
    losses = list()
    corrects = list()

    for step, (input_ids, labels) in enumerate(datasets):   #step은 미니 배치의순서, (inout_ids, labels) 배치단위의 데이터
        input_ids = input_ids.to(device)
        labels = labels.to(device).unsqueeze(1)

        logits = model(input_ids)   #예측값
        loss = criterion(logits, labels)
        losses.append(loss.item())
        yhat = torch.sigmoid(logits)>.5   # 0.5보다 크면 1(양성), 작으면 0(음성)
        corrects.extend(torch.eq(yhat, labels).cpu().tolist())        #예측값과 실제값이 같은지비교

    print(f"Val Loss : {np.mean(losses)}, Val Accuracy : {np.mean(corrects)}")


In [17]:
epochs = 5
interval = 500   #500번 미니배치마다 손실출력

for epoch in range(epochs):
    train(classifier, train_loader, criterion, optimizer, device, interval)
    test(classifier, test_loader, criterion, device)

Train Loss 0 : 0.6930583715438843
Train Loss 500 : 0.6919680641082946
Train Loss 1000 : 0.6585782636831571
Train Loss 1500 : 0.6291041598567797
Train Loss 2000 : 0.6040527998045765
Train Loss 2500 : 0.5838019549548745
Val Loss : 0.4860045122928894, Val Accuracy : 0.7602
Train Loss 0 : 0.5803261995315552
Train Loss 500 : 0.46590390563487055
Train Loss 1000 : 0.4630418364237715
Train Loss 1500 : 0.4528295576751113
Train Loss 2000 : 0.44626679845612266
Train Loss 2500 : 0.44209505689115536
Val Loss : 0.4233264391557477, Val Accuracy : 0.8052
Train Loss 0 : 0.5992528796195984
Train Loss 500 : 0.37632469936997115
Train Loss 1000 : 0.3788777944314611
Train Loss 1500 : 0.38230555796925025
Train Loss 2000 : 0.3822521733260643
Train Loss 2500 : 0.3789940705178071
Val Loss : 0.40160718850624827, Val Accuracy : 0.8174
Train Loss 0 : 0.14265784621238708
Train Loss 500 : 0.33898823662194427
Train Loss 1000 : 0.3346739466842655
Train Loss 1500 : 0.3360012321572237
Train Loss 2000 : 0.337791954663799

In [18]:
## 학습된 모델로부터 임베딩 추출
toekn_to_embedding = dict()
embedding_matrix = classifier.embedding.weight.detach().cpu().numpy()  # 임베딩 레이어에서 가중치를 가져옴   , detach는 추론 모드로 임베딩 가중치를 가져오기 위해, 그래디언트 계산을 차단합니다.

for word, emb in zip(vocab, embedding_matrix): # vocab과 embedding_matrix를 동시에 순회, vocab에는 단어, embedding_matrix에는 임베딩벡터
    toekn_to_embedding[word] = emb

token = vocab[1000]
print(token, toekn_to_embedding[token])

보고싶다 [ 2.81532854e-01 -6.03298604e-01 -2.26492539e-01 -1.58868110e+00
 -1.02817498e-01 -7.86689103e-01 -1.99850976e+00  1.22364545e+00
  1.56242937e-01  1.33723676e+00  1.64831951e-01  2.67705560e-01
  4.02862728e-01  9.92026567e-01 -8.92821252e-02  1.10149610e+00
  1.84162199e-01  6.70502335e-03 -2.05980942e-01  4.68503863e-01
 -6.60967052e-01 -1.15528345e+00 -1.52757192e+00 -1.03235078e+00
  1.80954707e+00  6.85172677e-01 -6.97342575e-01 -4.21210557e-01
 -5.63878044e-02  2.52857471e+00 -5.10574043e-01  1.27529049e+00
 -1.59229267e+00  8.77205670e-01  4.58272427e-01  1.37642130e-01
  1.05171287e+00  7.64356017e-01  3.98507684e-01 -9.84263420e-01
  7.35568181e-02 -2.07504243e-01  2.30032504e-01  1.39554656e+00
  4.45250750e-01  2.03182817e+00 -1.76673341e+00  2.18037099e-01
  7.69549608e-01  1.48528588e+00  1.53258121e+00 -1.49295494e-01
  5.88497341e-01  2.56121945e+00  3.30616534e-03  1.40057838e+00
  6.24860823e-01 -1.89067185e-01  2.10599041e+00 -1.20676410e+00
 -2.13464618e+00 -9.

In [19]:
## 사전학습된 모델로 임베딩 계층 초기화
from gensim.models import Word2Vec

word2vec = Word2Vec.load("word2vec.model")
init_embeddings = np.zeros((n_vocab, embedding_dim))

for index, token in id_to_token.items():
    if token not in ["<pad>", "<unk>"]:
        init_embeddings[index] = word2vec.wv[token]

embedding_layer = nn.Embedding.from_pretrained(torch.tensor(init_embeddings, dtype=torch.float32))

In [23]:
# 사전 학습된 임베딩 계층 적용:

class SentenceClassifier(nn.Module):
    def __init__(
        self,
        n_vocab,
        hidden_dim,
        embedding_dim,
        n_layers,
        dropout=0.5,
        bidirectional=True,
        model_type="lstm",
        pretrained_embedding=None
    ):
        super().__init__()
        if pretrained_embedding is not None:
            self.embedding = nn.Embedding.from_pretrained(
                torch.tensor(pretrained_embedding, dtype=torch.float32)
            )
        else:
            self.embedding = nn.Embedding(
                num_embeddings=n_vocab,
                embedding_dim=embedding_dim,
                padding_idx=0
            )
        
        if model_type == "rnn":
            self.model = nn.RNN(
                input_size=embedding_dim,
                hidden_size=hidden_dim,
                num_layers=n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True,
            )
        elif model_type == "lstm":
            self.model = nn.LSTM(
                input_size=embedding_dim,
                hidden_size=hidden_dim,
                num_layers=n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True,
            )

        if bidirectional:
            self.classifier = nn.Linear(hidden_dim * 2, 1)
        else:
            self.classifier = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        output, _ = self.model(embeddings)
        last_output = output[:, -1, :]
        last_output = self.dropout(last_output)
        logits = self.classifier(last_output)
        return logits

In [24]:
classifier = SentenceClassifier(n_vocab = n_vocab, hidden_dim=hidden_dim, embedding_dim=embedding_dim, n_layers=n_layers, pretrained_embedding=init_embeddings).to(device)
criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = optim.RMSprop(classifier.parameters(), lr=0.001)

In [25]:
epochs = 5
interval = 500

for epoch in range(epochs):
    train(classifier, train_loader, criterion, optimizer, device, interval)
    test(classifier, test_loader, criterion, device)

Train Loss 0 : 0.6865137219429016
Train Loss 500 : 0.6893818608063186
Train Loss 1000 : 0.63530323777225
Train Loss 1500 : 0.5967390515818586
Train Loss 2000 : 0.5754937220742737
Train Loss 2500 : 0.5604580375956135
Val Loss : 0.4825119415220742, Val Accuracy : 0.7702
Train Loss 0 : 0.42515647411346436
Train Loss 500 : 0.49300900262273
Train Loss 1000 : 0.4861773193537534
Train Loss 1500 : 0.4804233860604212
Train Loss 2000 : 0.47649934870758276
Train Loss 2500 : 0.474079876786611
Val Loss : 0.45986667418251403, Val Accuracy : 0.7924
Train Loss 0 : 0.4736987352371216
Train Loss 500 : 0.45287042436842434
Train Loss 1000 : 0.45686919006196175
Train Loss 1500 : 0.45499423500659225
Train Loss 2000 : 0.4518256302083331
Train Loss 2500 : 0.4515298610387064
Val Loss : 0.4375054622515322, Val Accuracy : 0.7984
Train Loss 0 : 0.670641303062439
Train Loss 500 : 0.44490503009683835
Train Loss 1000 : 0.4386521572029436
Train Loss 1500 : 0.441603958100418
Train Loss 2000 : 0.4406450189378249
Train 